In [21]:
import numpy as np
import pyvista as pv
import SALib.sample.sobol as sobol_sample
import SALib.sample.morris as morris_sample
from SALib.analyze import sobol, morris
import sys
from pathlib import Path
import pint
import pandas as pd
from pint_pandas import PintArray, PintType
import torch

In [2]:
sys.path.append(str(Path.cwd().parent))

In [ ]:
from src.offline_stage import NirbModule,Normalizations
from src.pod import MeanNormalizer, MinMaxNormalizer, Standardizer
from src.utils import (load_pint_data,
                       safe_parse_quantity,
                       format_quantity)

### Meta Data

In [4]:
PARAMETER_SPACE = "01"
ACCURACY = 1e-5
ROOT = Path.cwd().parent / "data" / PARAMETER_SPACE
ROOT.exists()
IS_SAVE_PNG = True
SUFFIX = "mean"
ureg = pint.get_application_registry()

In [5]:
# choose normalizer form SUFFIX
if "mean" in SUFFIX.lower():
    scaling = Normalizations.Mean
elif "min_max" in SUFFIX.lower():
    scaling = Normalizations.MinMax
else:
    raise ValueError("Invalid suffix.")

### Import Neural Network

In [6]:
chk_pt_paths = [path for path in (ROOT / f"nn_logs_{ACCURACY:.1e}{SUFFIX}").rglob("*.ckpt")]
chk_pt_paths = sorted(chk_pt_paths, key = lambda p: p.parent.parent.stem)
print("Please choose checkpoint by index in the next cell.")
for idx, path in enumerate(chk_pt_paths):
    print(f"{idx:02d} ({path.parent.parent.stem}): {path.name}")

Please choose checkpoint by index in the next cell.
00 (version_0): epoch=199999-step=200000.ckpt
01 (version_1): epoch=399999-step=400000.ckpt
02 (version_10): epoch=99999-step=200000.ckpt
03 (version_11): epoch=24999-step=50000.ckpt
04 (version_12): epoch=599-step=1200.ckpt
05 (version_13): epoch=6999-step=49000.ckpt
06 (version_14): epoch=6999-step=49000.ckpt
07 (version_15): epoch=14999-step=60000.ckpt
08 (version_16): epoch=7399-step=29600.ckpt
09 (version_17): epoch=7499-step=30000.ckpt
10 (version_18): epoch=17499-step=70000.ckpt
11 (version_19): epoch=22499-step=90000.ckpt
12 (version_2): epoch=16147-step=16148.ckpt
13 (version_20): epoch=9999-step=50000.ckpt
14 (version_3): epoch=199999-step=200000.ckpt
15 (version_4): epoch=34328-step=34329.ckpt
16 (version_5): epoch=399999-step=400000.ckpt
17 (version_6): epoch=599999-step=600000.ckpt
18 (version_7): epoch=99999-step=100000.ckpt
19 (version_8): epoch=149999-step=150000.ckpt


In [7]:
chk_pt_path = chk_pt_paths[10]
print("Selected following checkpoint:")
print(chk_pt_path.relative_to(chk_pt_path.parents[2]))

Selected following checkpoint:
version_18/checkpoints/epoch=17499-step=70000.ckpt


In [19]:
coefficient_model = NirbModule.load_from_checkpoint(chk_pt_path)
coefficient_model = coefficient_model.to('cpu')
coefficient_model.eval()

NirbModule(
  (loss): MSELoss()
  (activation): Sigmoid()
  (model): NIRB_NN(
    (layers): Sequential(
      (0): Linear(in_features=2, out_features=95, bias=True)
      (1): Sigmoid()
      (2): Linear(in_features=95, out_features=228, bias=True)
      (3): Sigmoid()
      (4): Linear(in_features=228, out_features=204, bias=True)
      (5): Sigmoid()
      (6): Linear(in_features=204, out_features=106, bias=True)
      (7): Sigmoid()
      (8): Linear(in_features=106, out_features=92, bias=True)
      (9): Sigmoid()
      (10): Linear(in_features=92, out_features=23, bias=True)
    )
  )
  (msa_metric): MeanAbsoluteError()
)

### Import Parameters and Training Snapshots

In [8]:
parameters = load_pint_data(ROOT / "Training_samples.csv")
print("Following Parameters can be varied:")
min_max_data = parameters.pint.dequantify().agg(["min", "max"])
print(min_max_data)
problem = dict(
    num_vars = len(parameters.columns),
    names = list(parameters.columns)
)

df_problem = pd.DataFrame(
    {col_name : PintArray(pd.Series([0, 0]), dtype=unit) for col_name, unit in zip(parameters.columns, parameters.dtypes)},
    index=["min", "max"])
# df_problem.index = parameters.columns
print("\nDefined problem for SA")
print(problem)
print(df_problem)

Following Parameters can be varied:
            host_k    T_h
unit    meter ** 2 kelvin
min   1.035142e-18  403.6
max   9.660509e-16  492.7

Defined problem for SA
{'num_vars': 2, 'names': ['host_k', 'T_h']}
     host_k  T_h
min     0.0  0.0
max     0.0  0.0


In [9]:
for col_name, dytpe in zip(parameters.columns, parameters.dtypes):
    print(col_name)
    print(dytpe)
df_problem = pd.DataFrame(
    {col_name : PintArray(pd.Series([0, 0]), dtype=unit) for col_name, unit in zip(parameters.columns, parameters.dtypes)}
    )#
df_problem

host_k
pint[meter ** 2][float64]
T_h
pint[kelvin][float64]


,host_k,T_h
0,0.0,0.0
1,0.0,0.0


In [10]:
control_mesh_suffix =  "s100_100_100_b0_4000_0_5000_-4000_0"
parameters = parameters.pint.dequantify().to_numpy()
basis_functions             = np.load(ROOT / "TrainingMapped" / control_mesh_suffix / "BasisFunctions" / f"basis_fts_matrix_{ACCURACY:.1e}{SUFFIX}.npy")
training_snapshots_npy      = np.load(ROOT / "TrainingMapped" / control_mesh_suffix / "Exports" / "Training_temperatures.npy")
training_snapshots_npy      = training_snapshots_npy[:, -1, :] #! last time step

In [11]:
print(basis_functions.shape)
print(parameters.shape)
print(training_snapshots_npy.shape)

(23, 20161)
(200, 2)
(160, 20161)


## Define the problem

In [12]:
print(problem)
print(min_max_data)

{'num_vars': 2, 'names': ['host_k', 'T_h']}
            host_k    T_h
unit    meter ** 2 kelvin
min   1.035142e-18  403.6
max   9.660509e-16  492.7


In [13]:
df_problem.loc["min", "host_k"] = 1e-17
df_problem.loc["max", "host_k"] = 9e-16
df_problem.loc["min", "T_h"] = ureg.Quantity(140, ureg.degC)
df_problem.loc["max", "T_h"] = ureg.Quantity(210, ureg.degC)
df_problem

,host_k,T_h
0,0.0,0.0
1,0.0,0.0
min,1e-17,413.15
max,9e-16,483.15


In [14]:
def get_min_max(col):
    min_val = df_problem.loc['min', col].magnitude
    max_val = df_problem.loc['max', col].magnitude
    assert min_val <= max_val, f"{col}: min {min_val} > max {max_val}"
    return [min_val, max_val]

problem["bounds"] = [get_min_max(col) for col in df_problem.columns]
problem

{'num_vars': 2,
 'names': ['host_k', 'T_h'],
 'bounds': [[np.float64(1e-17), np.float64(9e-16)],
  [np.float64(413.15), np.float64(483.15)]]}

## Sobol

In [15]:
param_values = sobol_sample.sample(problem, 131072) #2**17=131072
param_values

array([[5.81726210e-16, 4.49980815e+02],
       [8.33303515e-16, 4.49980815e+02],
       [5.81726210e-16, 4.67610615e+02],
       ...,
       [5.81720908e-16, 4.76451295e+02],
       [1.78921551e-16, 4.33580675e+02],
       [1.78921551e-16, 4.76451295e+02]], shape=(786432, 2))

In [17]:
standaridzer = Standardizer()
param_values = standaridzer.normalize(np.asarray(param_values))

In [ ]:
Y_0 = np.zeros(len(param_values))
param_t = torch.from_numpy(param_values.astype(np.float32))
param_t

tensor([[ 0.4932,  0.0906],
        [ 1.4725,  0.0906],
        [ 0.4932,  0.9631],
        ...,
        [ 0.4932,  1.4006],
        [-1.0746, -0.7210],
        [-1.0746,  1.4006]])

In [ ]:
Y_0 = np.zeros(len(param_values))
RB_ouptuts = coefficient_model(param_t).detach().numpy()
for i in range(len(param_values)):
    RB_full_metric=np.mean(np.matmul(RB_ouptuts[i,:], basis_functions))
    Y_0[i]=RB_full_metric   


In [32]:
Y_0

array([ 0.00260833,  0.00238769,  0.06223905, ...,  0.09744677,
       -0.03782518,  0.08723523], shape=(786432,))